In [ ]:
!pip3 install kaggle

In [ ]:
!mkdir .kaggle

In [ ]:
!mkdir ~/.kaggle/

In [ ]:
import json
token = {"username":"sudeeprajput21","key":"1766481fd53783ab03477079054a9b6c"}
with open('/content/.kaggle/kaggle.json','w') as file:
  json.dump(token, file)

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

Change permission.


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d srbhshinde/flickr8k-sau

In [ ]:
!kaggle datasets download -d yliu9999/glove6b50d

In [ ]:
!cd /content/{/content}/datasets 

In [ ]:
!unzip /content/{/content}/datasets/yliu9999/glove6b50d/glove6b50d.zip

In [ ]:
!unzip /content/{/content}/datasets/srbhshinde/flickr8k-sau/flickr8k-sau.zip

In [ ]:
ls

In [ ]:
cd Flickr_Data/

In [ ]:
ls

In [ ]:
cd ..

In [ ]:
flickr_path='Flickr_Data/'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords                 # will collect stopwords for english language
import string
import json
from time import time
import pickle                                     # helps storing different datatypes like list, arrays & helps serializing them
from keras.applications.vgg16 import VGG16        # pretrained model for CNN
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions          #pretrained model for RNN
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical             # for categorical division
from keras.layers import Input, Dense, Dropout, Embedding,LSTM
from keras.layers.merge import add
import tensorflow as tf

In [ ]:
with open(flickr_path + "Flickr_TextData/Flickr8k.token.txt") as filepath:
  captions = filepath.read()
  filepath.close()

In [ ]:
captions = captions.split ("\n") [:-1]

In [ ]:
len(captions)

In [ ]:
captions[:5]

- Creating a dictionary where key is 'img_name' & value is list of captions 

In [ ]:
 descriptions = {}                          # description contains the name of the image in jpg

for element in captions:
  img_to_cap = element.split("\t")
  img_name = img_to_cap[0].split(".")[0]
  cap = img_to_cap[1]

  if descriptions.get(img_name) == None:
    descriptions[img_name] = []

  descriptions[img_name].append(cap)

In [ ]:
descriptions['1000268201_693b08cb0e']

**Data Cleaning**
- Lower each word.
- Remove punctuations.
- Remove words less than length 1.

In [ ]:
def clean_text(sample):                                   
  sample = sample.lower()

  sample = re.sub("[^a-z]+"," ", sample)

  sample = sample.split()

  sample = [s for s in sample if len(s)>1]

  sample = " ".join(sample)

  return sample

In [ ]:
clean_text ("I am Sudeep 21 03 rAjput @1996")

In [ ]:
# Modify all the captions (cleaned captions)

for key, desc_list in descriptions.items():
  for i in range (len(desc_list)):
    desc_list[i] = clean_text(desc_list[i])

In [ ]:
descriptions ['1000268201_693b08cb0e']

In [ ]:
# Writing clean descriptions to .txt file

f = open("descriptions.txt","w")
f.write(str(descriptions))
f.close()

In [ ]:
# Reading descriptions file to check captions

f = open("descriptions.txt","r")
descriptions = f.read()
f.close()

json_acceptable_string = descriptions.replace("'","\"")
descriptions = json.loads(json_acceptable_string)

In [ ]:
descriptions

In [ ]:
# Finding unique vocabulary (finding how many unique words are present in 40K captions)

vocabulary = set()

for key in descriptions.keys():
  [vocabulary.update(i.split()) for i in descriptions[key]]

print('Vocabulary Size:%d' %len(vocabulary))







In [ ]:
# Write all words in description dictionary

all_vocab = []

for key in descriptions.keys():
  [all_vocab.append(i) for des in descriptions[key] for i in des.split()]

print ('Vocabulary Size:%d' %len(all_vocab))
print (all_vocab[:15])



- Count the frequency of each word, sort them & discard the words having frequency lesser than the threshold value

In [ ]:
import collections                                      # Create a vocabulary which will have atleast 10 times the word is occuring in entire process
                                                        # Create a threshold for it.
counter = collections.Counter(all_vocab)

dict_ = dict(counter)

threshold_value = 10

sorted_dict = sorted(dict_.items(), reverse = True, key = lambda x: x[1])
sorted_dict = [x for x in sorted_dict if x[1] > threshold_value]
all_vocab = [x[0] for x in sorted_dict]



In [ ]:
len(all_vocab)

1845

- Now we have trained .txt file so we will train image file. Open the file & read it.

In [ ]:
# Train image file

f = open (flickr_path + "Flickr_TextData/Flickr_8k.trainImages.txt")
train = f.read()
f.close()

In [ ]:
train

In [ ]:
train = [e.split(".")[0] for e in train.split("\n")[:-1]]

In [ ]:
# Test image file (Reading test image file)

f = open (flickr_path + "Flickr_TextData/Flickr_8k.testImages.txt")
test = f.read()
f.close()

In [ ]:
test

In [ ]:
test = [e.split(".")[0] for e in test.split("\n")[:-1]]

- Create train descriptions dictionary which will be similar to earlier one but having only train samples
- Add startseq + endseq
- Training have different dict. & testing have different dict. Now we load description from description.txt which was saved on hard disk.

In [ ]:
train_descriptions = {}

for t in train:
  train_descriptions[t] = []

  for cap in descriptions[t]:
    cap_to_append = "startseq " + cap + " endseq"         
    train_descriptions[t].append (cap_to_append)

In [ ]:
# Mapping descriptions to images

train_descriptions['1000268201_693b08cb0e']

In [ ]:
images = flickr_path = "Images/"

**Model for preprocessing**
- ResNet has all the information about images, it is a pretrained transfer learning model.

In [ ]:
model = ResNet50(weights="imagenet", input_shape=(224,224,3))

In [ ]:
model.summary()

- Create a new model by removing the last layer(output layer of 1000 classes) from resnet50

In [ ]:
model_new = Model(model.input, model.layers[-2].output)

In [ ]:
model_new.summary()

- This model was trained on the image dataset i.e. 1000 different classes of images. we removed the last layer to get rid of bottleneck features.

-- Now preprocessing images & converting my images to arrays then i did dimensioning and after that converting them to matrices.

In [ ]:
def preprocess_image(img):
  img = image.load_img(img, target_size=(224,224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  return img


In [ ]:
def encode_image(img):
  img = preprocess_image(img)
  feature_vector = model_new.predict(img)
  feature_vector = feature_vector.reshape(feature_vector.shape[1])
  return feature_vector

In [ ]:
# Encoding each image in train set using Resnet

encoding_train = {}

for index, img in enumerate(train):

  img = "/content/Flickr_Data/Images/{}.jpg".format(train[index])
  encoding_train[img[len(images):]] = encode_image(img)

  if index%100==0:
        print ("Encoding image-" + str(index))

- Encoding the images so they are basically given a number to it. And save the training features to the disk.

In [ ]:
# Save the train features to disk

with open ("./encoded_train_images.pkl", "wb") as encoded_pickle:
   pickle.dump(encoding_train, encoded_pickle)

In [ ]:
# Encoding test images

encoding_test = {}

for index, img in enumerate(test):

  img = "/content/Flickr_Data/Images/{}.jpg".format(test[index])
  encoding_test[img[len(images):]] = encode_image(img)

  if index%100==0:
        print ("Encoding image-" + str(index))

In [ ]:
# Save the bottleneck train features to disk

with open ("./encoded_test_images.pkl", "wb") as encoded_pickle:
  pickle.dump(encoding_test, encoded_pickle)

In [ ]:
# Load the train images features from disk

with open ("./encoded_train_images.pkl", "rb") as encoded_pickle:
  encoding_train=pickle.load(encoded_pickle)

In [ ]:
encoding_train['2513260012_03d33305cf.jpg']

In [ ]:
# Load the test image features from disk

with open ("./encoded_test_images.pkl", "rb") as encoded_pickle:
  encoding_test = pickle.load(encoded_pickle)

- Now I have to data process my captions. Captions are something we want to predict so during training period caption will be target variable for our model but the prediction of entire caption given the images does not happen all at once, so we will be predicting the captions word by word.

- One word is generated then RNN is going to generate oter word based on the sequence it will follow.

In [ ]:
# word_to_index is mapping between each unique word in all_vocab to int value & index_to_word is vice versa.

index = 1
word_to_index = {}
index_to_word = {}


for e in all_vocab:
  word_to_index[e] = index
  index_to_word[index] = e
  index+= 1 


In [ ]:
word_to_index ['above']

In [ ]:
index_to_word [225]

In [ ]:
len(word_to_index)

In [ ]:
# Adding startseq & endseq. Our machine doesn't understand words so convert these to indexes.

word_to_index ['startseq'] = 1846
word_to_index ['endseq'] = 1847

index_to_word [1846] = 'startseq'
index_to_word [1847] = 'endseq'



In [ ]:
# vocab_size is total vocabulary len +1 because we will append 0's as well

vocab_size = len(index_to_word)+1
print(vocab_size)

In [ ]:
#Now again making list of everything & storing max length to each one of them.

all_captions_len = []

for key in train_descriptions.keys():
    for cap in train_descriptions[key]:
        all_captions_len.append(len(cap.split()))

max_len = max(all_captions_len)
print(max_len)

- Now using Data Generator because caption has to be generated through images. I am giving image so it has to create a sequence of words which will be able to find what exact caption to be given.

In [ ]:
def data_generator (train_descriptions, encoding_train, word_to_index, max_len, num_photos_per_batch):

  x1, x2, y=[], [], []
  n = 0

  while True:

    for key, desc_list in train_descriptions.items():
      n += 1

      photo = encoding_train[key+".jpg"]

      for desc in desc_list:

        seq = [word_to_index[word] for word in desc.split() if word in word_to_index]

        for i in range(1,len(seq)):

          in_seq = seq[0:i]
          out_seq = seq[i]

          in_seq = pad_sequences ([in_seq], maxlen = max_len, value = 0, padding = 'post')[0]          # adding 0 padding because I need to start from 0.

          out_seq = to_categorical ([out_seq], num_classes = vocab_size)[0]

          x1.append(photo)
          x2.append(in_seq)
          y.append(out_seq)

      if n == num_photos_per_batch:                                      # It will start batch processing part of it now.
        yield [[np.array(x1), np.array(x2)], np.array(y)]                # Also it will take every caption as a batch.
        x1, x2, y = [], [], []
        n=0

In [ ]:
f = open("glove.6B.50d.txt", encoding = 'utf8')
f

- Creating dictionary for embedding means creating vector for words that are created.

In [ ]:
embedding_index = {}

for line in f:
  values = line.split()
  word =  values[0]
  Coefs = np.array(values[1: ], dtype="float")

  embedding_index[word] = Coefs
f.close()

In [ ]:
# Converting words into vector directly - (Embedding Layer Output)

def get_embedding_output():
  emb_dim = 50
  embedding_output = np.zeros((vocab_size, emb_dim))

  for word, idx in word_to_index.items():
    embedding_vector = embedding_index.get(word)
  
    if embedding_vector is not None:
      embedding_output[idx] = embedding_vector
      
  return embedding_output

embedding_output = get_embedding_output()

In [ ]:
embedding_output

In [ ]:
embedding_output.shape                    # checking shape of output.

**Defining Model**

- Since the input consists of 2 parts, an image vector & partial caption, we cannot use sequential model so we create custom.

- Features are extracted through CNN. Now we are going to decode everything. So every image we see, the objects are identified, the features are extracted from CNN and RNN will be decoding each and every image.

In [ ]:
# Image feature extraction model, inputs image feature vector

input_img_feat = Input(shape=(2048))
inp_img1 = Dropout(0.3)(input_img_feat)
inp_img2 = Dense(256, activation='relu')(inp_img1)

In [ ]:
# Partial caption sequence model, inputs captions

input_cap = Input(shape = max_len)
inp_cap1 = Embedding(input_dim= vocab_size, output_dim= 50, mask_zero= True)(input_cap)
inp_cap2 = Dropout(0.3)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

In [ ]:
decoder1 = add([inp_img2,inp_cap3])
decoder2 = Dense(256, activation= 'relu')(decoder1)
outputs = Dense(vocab_size, activation= 'softmax')(decoder2)

# Merge two networks
model = Model(inputs = [input_img_feat, input_cap], outputs = outputs)

In [ ]:
model.summary()

In [ ]:
model.layers[2].set_weights([embedding_output])
model.layers[2].trainable = False

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam")

In [ ]:
epochs = 20                                   # Hyperparameter is the epoch.
number_pics_per_batch = 3
steps = len(train_descriptions)               # what steps to be taken. Step size is also learning rate.

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_index, max_len, number_pics_per_batch)

    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

    model.save('./model_' + str(i) + '.h5')

In [ ]:
# Taking model from after 19th epoch

model = model("./model_19.h5")

- Creating a function for predicting captions. Image I am giving will be converted to word, then word to index & finally it will be converted to the caption. It will be splited and I have maintained all the ratio of it.

In [ ]:
def predict_caption(photo):
  in_text = "startseq"

  for i in range(max_len):
    sequence = [word_to_index[w] for w in in_text.split() if w in word_to_index]
    sequence = pad_sequences([sequence], maxlen = max_len, padding = 'post')

    ypred = model.predict([photo, sequence])
    ypred = ypred.argmax()
    word = index_to_word[ypred]
    in_text = ' ' + word

    if word = 'endseq':
      break

   final_caption = in_text.split()
   final_caption = final_caption[1 :-1]
   final_caption = ' '.join(final_caption)

   return final_caption

In [ ]:
for i in range(20):                           # Doing it for the first 20 images.
  rn = np.random.randint(0, 1000)
  img_name = list(encoding_test.keys())[rn]
  photo = encoding_test[img_name].reshape((1, 2048))

  i = plt.imread(images + img_name)
  plt.imshow(i)
  plt.axis("off")
  plt.show()

  caption = predict_caption(photo)
  print(caption)